In [ ]:
swid = ""
espn_s2 = ""
league_id = ""
season = 2023
week = 1
home_team = ""
away_team = ""
start_date = ""
end_date = ""
use_current_score = False
stat_type = "season average"

home_team_add = ""
home_team_remove = ""
away_team_add = ""
away_team_remove = ""

# App description

This app helps the user to prepare for an upcoming match-up between two fantasy teams. This is based on a league with *Head to Head Each Category* scoring type and 9 statistical categories (FG%, FT%, 3PM, REB, AST, STL, BLK, TO, PTS).

* It compares their schedule (number of starter players and unused players) 
* It compares the teams' historic stats up to this round
* Simulates/projects the match-up based on the players' average stats and schedule. 
* Allows for scenarios, such as replace player X with player Y

Further details on [this Medium blog post](https://g-giasemidis.medium.com/nba-fantasy-analytics-with-python-on-epsn-f03f10a60187).

Use this *public* league id `10149515` for trying out app. No need for `swid` and `espn_s2` cookies. This league is based on the same nine aforementioned stats, but uses a *Head to Head Points* scoring system. Here, the league is emulated as if the scoring system was "Head to Head Each Category". Checkout the [Post round analysis](https://espn-nba-fantasy.herokuapp.com/) app for the participated teams and their abbreaviations.

Report bugs and issues [here](https://github.com/giasemidis/espn-nba-fantasy/issues).


## App parameters

* `swid`: If your league is public, leave blank. Otherwise: In Safari these cookies can be found by opening the Storage tab of Developer tools (developer tools can be turned on in preferences), and looking under espn.com in the Cookies folder. In Chrome, they can be found in Settings -> Cookies and other site data -> See all cookies and site data, and search for ESPN.
* `espn_s2`: If your league is public, leave blank. Otherwise, follow above instructions. **CAUTION:** It is likely the cookie id to contain the `%` character. The corresponding field does not accept such a character. Replace it with a `!` (exclamation mark)
* `League ID`: Go to your ESPN league page. The URL contains `leagueId=123455`. Copy the number next to the `leagueid` parameters
* `Season`: Leave this to current season, i.e. 2023, unless you want to run analysis of a round of a previous season.
* `Round`: The round for analysis (a positive integer). It must be a future round, or a round in progress.
* `Home team`: Abbrebiation of the home team
* `Away team`: Abbreviation of the away team
* `Start date`: The start date of the analysis. If the round hasn't started, it should the start date of the round. If the round is in progress, it should be the current date. The format must be "YYYY-MM-DD"
* `End date`: The end data of the round. It must be later than the start date. The format must be "YYYY-MM-DD"
* `Use rounds stats`: If the round is in progress, use the accumulated stats so far to make the projection. I.e., if True, the projection will be the combination of the current stats and the projection stats for the remaining days. If False, the projection will be based only on the simulation results.
* `Stat type`: The type of stats for the simulation, i.e. "Season" (average season stats), "Last 30" (average of the last 30 days stats), etc.

The following fields should be used for exploring projections with different line-ups without having to make changes on ESPN. For example, questions like: "what if I replace player X with player Y for dates D1 and D2?" or "What if the oppent adds Player Z in their roster?"
* `Home team added players` (Optional): The players' names and dates that should be added in the home team's roster. It should have the following format. `Name A 2022-01-01, 2022-01-02, Name B`, where in this example we add to the player `Name A` on the roster for the two specified dates (comma separated with format `YYYY-MM-DD`) and the player `Name B` for the *all remaining dates* in the match-up (not dates added after the name).
* `Home team removed players` (Optional): The players' names and dates that should be removed from the home team's roster. Format same as the example above.
* `Away team added players` (Optional): Similar to `Home team added players`, but for the away team.
* `Away team removed players` (Optional): Similar to `Home team removed players`, but for the away team.

**Caution**: Injured players (status "O") and/or players in the IR are excluded from any calculations in the projections. If you want to include them (for any reason), add them in the roster as if they were not in.

Example: Say I want to add player "Cory Joseph" in my roster for only two dates until the end of the matchup and remove (replace) player "Terry Taylor" who is already in my team (home team). Later, I want to replace "Cory Joseph" with "Ish Smith" until the end of the match-up (2022-03-09).
I will do:
* "Home team added players" field: `Cory Joseph 2022-03-04, 2022-03-07, Ish Smith`
* "Home team added players" field: `Terry Taylor, Cory Joseph 2022-03-08, 2022-03-09`

In [ ]:
import sys
import pandas as pd

sys.path.append('../')
from src.EspnFantasyMatchUp import EspnFantasyMatchUp
from src.utils.io_json import read_json
from src.utils.app_utils import convert_input_strs_to_scn_dict, parameter_checks
from src.utils.get_logger import get_logger

In [ ]:
logger = get_logger(__name__)

In [ ]:
parameter_checks(logger, swid, espn_s2, league_id)

In [ ]:
cookies = {
    "swid": swid,
    "espn_s2": espn_s2.replace("!", "%")
}
league_settings = {
    "league_id": int(league_id),
    "season": season
}

In [ ]:
home_scn_players = convert_input_strs_to_scn_dict(home_team_add, home_team_remove)
away_scn_players = convert_input_strs_to_scn_dict(away_team_add, away_team_remove)

In [ ]:
logger.debug(home_scn_players)
logger.debug(away_scn_players)

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [ ]:
espn = EspnFantasyMatchUp(
    cookies, league_settings,
    week, 
    home_team=home_team, away_team=away_team, 
    start_date=start_date, end_date=end_date,
    home_scn_players=home_scn_players, away_scn_players=away_scn_players,
    stat_type=stat_type
)

# Head to head comparison of season historic stats

In [ ]:
espn.h2h_season_stats_comparison().astype("O")

# Compare Schedule (players per day)

In [ ]:
espn.compare_schedules().astype("O")

# Simulate Schedule

In [ ]:
espn.simulation(use_current_score=use_current_score)